# **Import the libararies**

In [ ]:
import pandas as pd
import numpy as np
import nltk as nlp
import re
import string


# **import the** **dataset**

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/Reassigment/nepalimoviereviews.csv.xlsx')

In [ ]:
df.info()

In [ ]:

from google.colab import drive
drive.mount('/content/drive')






# **Data preprocessing**

# **Removing punctuation**

In [ ]:
string.punctuation

In [ ]:
def remove_punctuation(text):
  addpunctuation = "।"
  string.punctuation = string.punctuation + addpunctuation
  punctuationfree="".join([i for i in text if i not in string.punctuation])
  return punctuationfree

In [ ]:
df['Reviews'] = df['Reviews'].apply(lambda x: remove_punctuation(x))

# **TOKENIZATION**

In [ ]:
! pip install nltk
! pip install nepalitokenizer
! pip install nepali-stemmer

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nepali_stemmer.stemmer import NepStemmer
from nepalitokenizer import NepaliTokenizer
nepstem = NepStemmer()
tokenize = NepaliTokenizer()

In [ ]:
import nltk
nltk.download('punkt')


In [ ]:
df['Reviews']= df['Reviews'].apply(word_tokenize)

In [ ]:
df['Reviews']

# **Removing the stopwords**

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')

In [ ]:
def remove_stopwords(review):
    stop_words = set(stopwords.words('nepali'))
    if isinstance(review, list): # Check if review is a list
        filtered_words = [word for word in review if word not in stop_words]
    else: # Assume it's a string if not a list
        filtered_words = [word for word in review.split() if word not in stop_words]
    return ' '.join(filtered_words)

In [ ]:
df['Reviews']= df["Reviews"].apply(remove_stopwords)


In [ ]:
df['Reviews'][0]

# **Steaming**

In [ ]:
def Stemming(text):
    # Stemming the nepali words
    stemmed_text = []
    for word in text.split():
        stemmed_text.append(nepstem.stem(word))
    text = ' '.join(stemmed_text)

    # Remove the leading and trailing spaces
    text = text.split()
    text = ' '.join(text)

    return text


In [ ]:
df['Reviews'] = df['Reviews'].apply(Stemming)

In [ ]:
df['Reviews'][0]

In [ ]:
df['Reviews']= df["Reviews"].apply(remove_stopwords)


In [ ]:
df['Reviews'][0]


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(binary=True, ngram_range=(1,2))
bow = cv.fit_transform(df['Reviews'])


In [ ]:
bow = cv.fit_transform(df['Reviews'])


In [ ]:
print(cv.vocabulary_)

In [ ]:
print (bow[0].toarray())

# TF-ID

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()

In [ ]:
tfidf_matrix = tfidf.fit_transform(df['Reviews']).toarray()


In [ ]:
y = df.iloc[:,-1].values


In [ ]:
tfidf_matrix

# **Spliting the dataset**

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(tfidf_matrix, y, test_size=0.1, random_state=11)

# **Importing the models**

In [ ]:
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB


In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
clf1= GaussianNB()
clf2 = MultinomialNB()
clf3 = BernoulliNB()
clf4 = RandomForestClassifier()
clf1.fit(x_train, y_train)
clf2.fit(x_train, y_train)
clf3.fit(x_train, y_train)
clf4.fit(x_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
y_pred1 = clf1.predict(x_test)
y_pred2 = clf2.predict(x_test)
y_pred3 = clf3.predict(x_test)
y_pred4 = clf4.predict(x_test)

In [ ]:
print("Gaussian:",accuracy_score(y_test, y_pred1))
print("Multinomial:",accuracy_score(y_test, y_pred2))
print("Bernauli:",accuracy_score(y_test, y_pred3))
print("RandomForest:",accuracy_score(y_test, y_pred4))

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print("Bernauli model ")
clf3.fit(x_train, y_train)
y_pred = clf3.predict(x_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Data
categories = ['Class 0', 'Class 1']
precision = [0.83, 0.82]
recall = [0.71, 0.90]
f1_score = [0.77, 0.86]

x = np.arange(len(categories))
width = 0.2

fig, ax = plt.subplots()
bar1 = ax.bar(x - width, precision, width, label='Precision')
bar2 = ax.bar(x, recall, width, label='Recall')
bar3 = ax.bar(x + width, f1_score, width, label='F1-Score')

# Labels and Title
ax.set_xlabel('Categories')
ax.set_ylabel('Scores')
ax.set_title('Evaluation Metrics by Class of bernauli')
ax.set_xticks(x)
ax.set_xticklabels(categories)
ax.legend()

plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Data
metrics = ['Accuracy', 'Macro Avg Precision', 'Macro Avg Recall', 'Macro Avg F1-Score', 'Weighted Avg Precision', 'Weighted Avg Recall', 'Weighted Avg F1-Score']
values = [0.82, 0.83, 0.81, 0.81, 0.82, 0.82, 0.82]

fig, ax = plt.subplots()
ax.barh(metrics, values, color='skyblue')
ax.set_xlabel('Scores')
ax.set_title('Overall Model Performance')

plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Data
metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
aspect_based_values = [0.740, 0.741, 0.740, 0.740]
our_model_values = [0.82, 0.83, 0.71, 0.77]

x = np.arange(len(metrics))
width = 0.3

fig, ax = plt.subplots()
bar1 = ax.bar(x - width/2, aspect_based_values, width, label='Aspect-Based Model')
bar2 = ax.bar(x + width/2, our_model_values, width, label='Developed  Model')

# Adding labels and title
ax.set_xlabel('Metrics')
ax.set_ylabel('Scores')
ax.set_title('Comparison of Model Performance Metrics')
ax.set_xticks(x)
ax.set_xticklabels(metrics)
ax.legend()

# Adding value labels on top of the bars
def add_labels(bars):
    for bar in bars:
        height = bar.get_height()
        ax.annotate('%.2f' % height,
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')

add_labels(bar1)
add_labels(bar2)

plt.show()


In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# Confusion matrix data
cm = np.array([[20, 8], [4, 36]])

fig, ax = plt.subplots()
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False, ax=ax)
ax.set_xlabel('Predicted Labels')
ax.set_ylabel('True Labels')
ax.set_title('Confusion Matrix')

# Set the tick labels
ax.set_yticklabels([ 'Positive', 'Negative'])

plt.show()


In [ ]:

def predict(nepali_text):

    if isinstance(nepali_text, str):
        nepali_text = [nepali_text]

    # Apply word_tokenize, remove_stopwords, and Stemming to each item in the list
    processed_text = []
    for text in nepali_text:
        tokens = NepaliTokenizer().tokenizer(text) # Use NepaliTokenizer
        tokens = remove_stopwords(" ".join(tokens))
        tokens = Stemming(tokens)
        tokens = remove_stopwords(tokens)
        processed_text.append("".join(tokens))

    # Transform the processed text using TF-IDF
    pred = tfidf.transform(processed_text)
    results = []
    for i in range(pred.shape[0]):
        if np.squeeze(clf3.predict(pred[i])) == 1:
            results.append(nepali_text[i] + ": Positive")
        else:
            results.append(nepali_text[i] + ": Negative")

    return results

In [ ]:
predict("यो चलचित्र धेरै नै बोरिंग थियो र कथा पनि कमजोर थियो।")